<a href="https://colab.research.google.com/github/aigonna/ML_Skills/blob/main/1_LightAutoML_%E4%BD%BF%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
!mkdir /content/data
os.chdir('/content/data')

In [3]:
!rm -f /root/.kaggle/kaggle.json
!mkdir /root/.kaggle/
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c titanic -p /content/data

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 6.01MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 26.2MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 54.8MB/s]


In [5]:
!cp -r /content/data /content/drive/Shareddrives/mm/kaggles/tatanic

In [6]:
!pip install -U lightautoml

     |████████████████████████████████| 262 kB 6.5 MB/s 
     |████████████████████████████████| 23.9 MB 28.5 MB/s 
     |████████████████████████████████| 2.6 MB 50.6 MB/s 
     |████████████████████████████████| 232 kB 56.4 MB/s 
     |████████████████████████████████| 67.4 MB 66 kB/s 
     |████████████████████████████████| 302 kB 63.1 MB/s 
     |████████████████████████████████| 98 kB 8.9 MB/s 
     |████████████████████████████████| 420 kB 60.5 MB/s 
     |████████████████████████████████| 204 kB 60.3 MB/s 
     |████████████████████████████████| 804.1 MB 2.6 kB/s 
     |████████████████████████████████| 1.2 MB 72.9 MB/s 
     |████████████████████████████████| 37.1 MB 122 kB/s 
     |████████████████████████████████| 9.1 MB 15.3 MB/s 
     |████████████████████████████████| 3.3 MB 58.5 MB/s 
     |████████████████████████████████| 895 kB 50.0 MB/s 
     |████████████████████████████████| 636 kB 74.8 MB/s 
     |████████████████████████████████| 164 kB 77.2 MB/s 
     |██████████

## 1.导入包

In [7]:
# Standard python libraries
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# Imports from LightAutoML package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

##2.导入数据

In [10]:
%%time
from pathlib import Path
data_dir = Path("/content/data")

train_data = pd.read_csv(data_dir/'train.csv')

CPU times: user 7.74 ms, sys: 0 ns, total: 7.74 ms
Wall time: 8.12 ms


In [11]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
test_data = pd.read_csv(data_dir/'test.csv')

In [14]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
sample = pd.read_csv(data_dir/'gender_submission.csv')

In [16]:
sample.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


##3.清理数据得到额外的特征

- Age 年龄
- Cabin 船舱号
- Embarked 登船港口
- Fare 票价
- Name 乘客姓名
- Parch 不同代直系亲属人数
- SibSp 同代直系亲属人数
- PassengerId 乘客ID
- Pclass 客舱等级
- Sex 性别
- Ticket 船票编号
- Survived 存活情况

In [52]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)#搜索字母后接.像Mr.这种敬称
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

def create_extra_features(data):
    data['Ticket_type'] = data['Ticket'].map(lambda x: x[0:3]) #Ticket_type只要Ticket前3字符
    data['Name_Words_Count'] = data['Name'].map(lambda x: len(x.split())) #name 词数
    #如果不nan就是有Cabin,转换为1,0int型数据
    data['Has_Cabin'] = data["Cabin"].map(lambda x: 1 - int(type(x) == float)) 
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    
    #根据票价的频率来选择箱子的均匀间隔，即每个箱子中含有的票价的数量是相同的
    data['CategoricalFare'] = pd.qcut(data['Fare'], 5).astype(str)
    #cut将年龄本身来选择箱子均匀间隔，即每个箱子的间距都是相同的
    data['CategoricalAge'] = pd.cut(data['Age'], 5).astype(str)
    
    data['Title'] = data['Name'].apply(get_title).replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    data['Title'] = data['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}).fillna(0)
    return data

In [53]:
train_data = create_extra_features(train_data)
test_data = create_extra_features(test_data)

##4.拆分数据集

In [56]:
%%time
import logging
tr_data, valid_data = train_test_split(train_data, 
                    test_size=0.2, 
                    stratify=train_data["Survived"], 
                    random_state=42)
logging.info('Parts sizes: tr_data = {}, valid_data = {}'
              .format(tr_data.shape, valid_data.shape))

CPU times: user 4.38 ms, sys: 2 µs, total: 4.39 ms
Wall time: 8.28 ms


##5.LightAutoML 使用

- Task

In [58]:
def f1_metric(y_true, y_pred):
    return f1_score(y_true, (y_pred > 0.5).astype(int))

task = Task('binary', metric = f1_metric)

In [59]:
%%time

roles = {'target': 'Survived',
         'drop': ['PassengerId', 'Name', 'Ticket']}

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [67]:
automl = TabularAutoML(task = task, 
            timeout = 600, # 训练最大时间600 seconds = 10 minutes
            cpu_limit = 4, # 使用cpu核心数Optimal for Kaggle kernels
            general_params = {'use_algos': [['lgb', 'lgb_tuned', 'cb']],})

In [68]:
oof_pred = automl.fit_predict(tr_data, roles = roles)
valid_pred = automl.predict(valid_data)

Start automl preset with listed constraints:
- time: 600 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (712, 19)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 591.3784086704254 secs
Start fitting Selector_LightGBM ...

===== Start working with fold 0 for Selector_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.463772	valid's Opt metric: 0.78
[200]	valid's binary_logloss: 0.415757	valid's Opt metric: 0.8
[300]	valid's binary_logloss: 0.398023	valid's Opt metric: 0.811321
[400]	valid's binary_logloss: 0.392941	valid's Opt metric: 0.811321
Early stopping, best iteration is:
[201]	valid's binary_logloss: 0.415784	valid's Opt metric: 0.811321
Selector_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====

Training until validation scores don't improve for 200 roun

INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-d24bd080-3319-4bb5-b71a-f148072120ea


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.428093	valid's Opt metric: 0.792453
[200]	valid's binary_logloss: 0.372543	valid's Opt metric: 0.788991
Early stopping, best iteration is:
[60]	valid's binary_logloss: 0.481989	valid's Opt metric: 0.808081
Lvl_0_Pipe_0_Mod_1_LightGBM fitting and predicting completed


INFO:optuna.study.study:Trial 0 finished with value: 0.8080808080808082 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07}. Best is trial 0 with value: 0.8080808080808082.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.465742	valid's Opt metric: 0.8
[200]	valid's binary_logloss: 0.396695	valid's Opt metric: 0.803738
[300]	valid's binary_logloss: 0.369033	valid's Opt metric: 0.803738
Early stopping, best iteration is:
[134]	valid's binary_logloss: 0.433807	valid's Opt metric: 0.811881
Lvl_0_Pipe_0_Mod_1_LightGBM fitting and predicting completed


...

Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.434758	valid's Opt metric: 0.776699
[200]	valid's binary_logloss: 0.373891	valid's Opt metric: 0.785047
Early stopping, best iteration is:
[75]	valid's binary_logloss: 0.463248	valid's Opt metric: 0.816327
Lvl_0_Pipe_0_Mod_1_LightGBM fitting and predicting completed


INFO:optuna.study.study:Trial 99 finished with value: 0.8163265306122448 and parameters: {'feature_fraction': 0.6770586265018569, 'num_leaves': 36, 'bagging_fraction': 0.704444297101984, 'min_sum_hessian_in_leaf': 0.0028975252964740035, 'reg_alpha': 0.0006320913135477741, 'reg_lambda': 0.09566511524552711}. Best is trial 21 with value: 0.8282828282828283.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.454764	valid's Opt metric: 0.8
[200]	valid's binary_logloss: 0.392104	valid's Opt metric: 0.773585
[300]	valid's binary_logloss: 0.367891	valid's Opt metric: 0.777778
Early stopping, best iteration is:
[117]	valid's binary_logloss: 0.438767	valid's Opt metric: 0.811881
Lvl_0_Pipe_0_Mod_1_LightGBM fitting and predicting completed


INFO:optuna.study.study:Trial 100 finished with value: 0.8118811881188119 and parameters: {'feature_fraction': 0.6520859576328109, 'num_leaves': 167, 'bagging_fraction': 0.6785344559896421, 'min_sum_hessian_in_leaf': 0.07965955444658947, 'reg_alpha': 0.18889991507181142, 'reg_lambda': 3.6266203364746765}. Best is trial 21 with value: 0.8282828282828283.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.338283	valid's Opt metric: 0.770642
Early stopping, best iteration is:
[16]	valid's binary_logloss: 0.448451	valid's Opt metric: 0.811881

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.418884	valid's Opt metric: 0.769231
Early stopping, best iteration is:
[61]	valid's binary_logloss: 0.410888	valid's Opt metric: 0.776699

===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.323167	valid's Opt metric: 0.78
Early stopping, best iteration is:
[41]	valid's binary_logloss: 0.343049	valid's Opt metric: 0.831683

===== Start working with fold 3 for Lvl_0_Pip

In [69]:
def acc_score(y_true, y_pred):
    return accuracy_score(y_true, (y_pred > 0.5).astype(int))
print('OOF acc: {}'.format(acc_score(tr_data['Survived'].values,      oof_pred.data[:, 0])))
print('VAL acc: {}'.format(acc_score(valid_data['Survived'].values, valid_pred.data[:, 0])))

OOF acc: 0.851123595505618
VAL acc: 0.8324022346368715


##6.用TabularUtilizedAutoML创建AutoML

In [70]:
automl = TabularUtilizedAutoML(task = task, 
            timeout = 600, # 600 seconds = 10 minutes
            cpu_limit = 4, # Optimal for Kaggle kernels
            general_params = {'use_algos': [['lgb', 'lgb_tuned', 'cb']],})

In [71]:
oof_pred = automl.fit_predict(tr_data, roles = roles)

Current random state: {'reader_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
Found general_params in kwargs, need to combine
Merged variant for general_params = {'use_algos': [['lgb', 'lgb_tuned', 'cb']], 'return_all_predictions': False}
Start automl preset with listed constraints:
- time: 599.9941513538361 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (712, 19)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 599.6551592350006 secs
Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.439669	valid's Opt metric: 0.8
[200]	valid's binary_logloss: 0.378879	valid's Opt metric: 0.777778
Early stopping, best iteration is:
[89]	valid's binary_logloss: 0.451325	valid's Opt metric: 0.816327

===== Start working with fold 1 for Lvl_0_Pipe_0_M

INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-365b1003-3c77-4028-82ae-f5d49256b6d2


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.428093	valid's Opt metric: 0.792453
[200]	valid's binary_logloss: 0.372543	valid's Opt metric: 0.788991
Early stopping, best iteration is:
[60]	valid's binary_logloss: 0.481989	valid's Opt metric: 0.808081
Lvl_0_Pipe_0_Mod_1_LightGBM fitting and predicting completed


INFO:optuna.study.study:Trial 0 finished with value: 0.8080808080808082 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07}. Best is trial 0 with value: 0.8080808080808082.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.465742	valid's Opt metric: 0.8
[200]	valid's binary_logloss: 0.396695	valid's Opt metric: 0.803738
[300]	valid's binary_logloss: 0.369033	valid's Opt metric: 0.803738
Early stopping, best iteration is:
[134]	valid's binary_logloss: 0.433807	valid's Opt metric: 0.811881
Lvl_0_Pipe_0_Mod_1_LightGBM fitting and predicting completed


...

Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.465211	valid's Opt metric: 0.742268
[200]	valid's binary_logloss: 0.41259	valid's Opt metric: 0.761905
[300]	valid's binary_logloss: 0.395714	valid's Opt metric: 0.807339
[400]	valid's binary_logloss: 0.388025	valid's Opt metric: 0.8
Early stopping, best iteration is:
[291]	valid's binary_logloss: 0.396196	valid's Opt metric: 0.807339

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.450072	valid's Opt metric: 0.755102
[200]	valid's binary_logloss: 0.3933	valid's Opt metric: 0.776699
[300]	valid's binary_logloss: 0.375578	valid's Opt metric: 0.833333
[400]	valid's binary_logloss: 0.368794	valid's Opt metric: 0.830189
[500]	valid's binary_logloss: 0.368844	valid's 

INFO:optuna.study.study:Trial 9 finished with value: 0.8045112781954886 and parameters: {'feature_fraction': 0.7989499894055425, 'num_leaves': 237, 'bagging_fraction': 0.5442462510259598, 'min_sum_hessian_in_leaf': 0.006080390190296602, 'reg_alpha': 2.5529693461039728e-08, 'reg_lambda': 8.471746987003668e-06}. Best is trial 7 with value: 0.8097928436911488.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.468837	valid's Opt metric: 0.752475
[200]	valid's binary_logloss: 0.412226	valid's Opt metric: 0.777778
[300]	valid's binary_logloss: 0.395821	valid's Opt metric: 0.788991
[400]	valid's binary_logloss: 0.387684	valid's Opt metric: 0.788991
Early stopping, best iteration is:
[227]	valid's binary_logloss: 0.404495	valid's Opt metric: 0.8

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.461094	valid's Opt metric: 0.75
[200]	valid's binary_logloss: 0.394137	valid's Opt metric: 0.780952
[300]	valid's binary_logloss: 0.371675	valid's Opt metric: 0.830189
[400]	valid's binary_logloss: 0.361352	valid's Opt metric: 0.830189
[500]	valid's binary_logloss: 0.359887	valid's O

INFO:optuna.study.study:Trial 10 finished with value: 0.8097928436911488 and parameters: {'feature_fraction': 0.5102569642516216, 'num_leaves': 152, 'bagging_fraction': 0.6843643863605486, 'min_sum_hessian_in_leaf': 0.024400771927662247, 'reg_alpha': 0.0031424128733854905, 'reg_lambda': 0.016301353379407572}. Best is trial 7 with value: 0.8097928436911488.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.388131	valid's Opt metric: 0.785047
[200]	valid's binary_logloss: 0.406459	valid's Opt metric: 0.769231
Early stopping, best iteration is:
[104]	valid's binary_logloss: 0.385892	valid's Opt metric: 0.785047

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.357491	valid's Opt metric: 0.819048
[200]	valid's binary_logloss: 0.374373	valid's Opt metric: 0.810811
Early stopping, best iteration is:
[105]	valid's binary_logloss: 0.355097	valid's Opt metric: 0.841121

===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.35813	valid's Opt metric: 0.8
Early stoppi

In [72]:
valid_pred = automl.predict(valid_data)

In [73]:
print('OOF acc: {}'.format(acc_score(tr_data['Survived'].values, oof_pred.data[:, 0])))
print('VAL acc: {}'.format(acc_score(valid_data['Survived'].values, valid_pred.data[:, 0])))

OOF acc: 0.8693820224719101
VAL acc: 0.8212290502793296


##7.训练整个数据集

In [74]:
automl = TabularUtilizedAutoML(task = task, 
                timeout = 600, # 600 seconds = 10 minutes
                cpu_limit = 4, # Optimal for Kaggle kernels
                general_params = {'use_algos': [['lgb', 'lgb_tuned', 'cb']],})
oof_pred = automl.fit_predict(train_data, roles = roles)
test_pred = automl.predict(test_data)

Current random state: {'reader_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
Found general_params in kwargs, need to combine
Merged variant for general_params = {'use_algos': [['lgb', 'lgb_tuned', 'cb']], 'return_all_predictions': False}
Start automl preset with listed constraints:
- time: 599.9908020496368 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (891, 19)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 599.6340253353119 secs
Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.457814	valid's Opt metric: 0.733333
[200]	valid's binary_logloss: 0.402691	valid's Opt metric: 0.747967
[300]	valid's binary_logloss: 0.382032	valid's Opt metric: 0.771654
[400]	valid's binary_logloss: 0.37441	valid's Opt metric: 0.758065
[500]	valid's

INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-9582dd1e-596f-4380-b4bb-f12d116f5cd7


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.442948	valid's Opt metric: 0.737705
[200]	valid's binary_logloss: 0.388189	valid's Opt metric: 0.761905
[300]	valid's binary_logloss: 0.373788	valid's Opt metric: 0.755906
[400]	valid's binary_logloss: 0.364127	valid's Opt metric: 0.777778
[500]	valid's binary_logloss: 0.364232	valid's Opt metric: 0.787402
Early stopping, best iteration is:
[386]	valid's binary_logloss: 0.363137	valid's Opt metric: 0.777778
Lvl_0_Pipe_0_Mod_1_LightGBM fitting and predicting completed


INFO:optuna.study.study:Trial 0 finished with value: 0.7777777777777778 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07}. Best is trial 0 with value: 0.7777777777777778.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.480899	valid's Opt metric: 0.722689
[200]	valid's binary_logloss: 0.417175	valid's Opt metric: 0.727273
[300]	valid's binary_logloss: 0.393334	valid's Opt metric: 0.741935
[400]	valid's binary_logloss: 0.382602	valid's Opt metric: 0.752
[500]	valid's binary_logloss: 0.375627	valid's Opt metric: 0.740157
Early stopping, best iteration is:
[339]	valid's binary_logloss: 0.388223	valid's Opt metric: 0.752
Lvl_0_Pipe_0_Mod_1_LightGBM fitting and predicting completed


...

Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.465111	valid's Opt metric: 0.707965
[200]	valid's binary_logloss: 0.418522	valid's Opt metric: 0.766667
[300]	valid's binary_logloss: 0.414002	valid's Opt metric: 0.780488
[400]	valid's binary_logloss: 0.424458	valid's Opt metric: 0.793651
Early stopping, best iteration is:
[264]	valid's binary_logloss: 0.41282	valid's Opt metric: 0.780488

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.443535	valid's Opt metric: 0.796748
[200]	valid's binary_logloss: 0.377357	valid's Opt metric: 0.8125
[300]	valid's binary_logloss: 0.352466	valid's Opt metric: 0.815385
[400]	valid's binary_logloss: 0.345057	valid's Opt metric: 0.824427
[500]	valid's binary_logloss: 0.34107	vali

INFO:optuna.study.study:Trial 6 finished with value: 0.7962674961119751 and parameters: {'feature_fraction': 0.6523068845866853, 'num_leaves': 39, 'bagging_fraction': 0.8421165132560784, 'min_sum_hessian_in_leaf': 0.057624872164786026, 'reg_alpha': 1.254134495897175e-07, 'reg_lambda': 0.00028614897264046574}. Best is trial 0 with value: 0.80370942812983.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.484478	valid's Opt metric: 0.666667
[200]	valid's binary_logloss: 0.431726	valid's Opt metric: 0.735043
[300]	valid's binary_logloss: 0.419019	valid's Opt metric: 0.756303
[400]	valid's binary_logloss: 0.419778	valid's Opt metric: 0.766667
[500]	valid's binary_logloss: 0.424742	valid's Opt metric: 0.786885
Early stopping, best iteration is:
[337]	valid's binary_logloss: 0.417459	valid's Opt metric: 0.766667

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's binary_logloss: 0.461883	valid's Opt metric: 0.774194
[200]	valid's binary_logloss: 0.398354	valid's Opt metric: 0.778626
[300]	valid's binary_logloss: 0.375835	valid's Opt metric: 0.787879
[400]	valid's binary_logloss: 0.362534	

INFO:optuna.study.study:Trial 7 finished with value: 0.784688995215311 and parameters: {'feature_fraction': 0.5171942605576092, 'num_leaves': 234, 'bagging_fraction': 0.6293899908000085, 'min_sum_hessian_in_leaf': 0.4467752817973907, 'reg_alpha': 6.388511557344611e-06, 'reg_lambda': 0.0004793052550782129}. Best is trial 0 with value: 0.80370942812983.


Start fitting Lvl_0_Pipe_0_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.446631	valid's Opt metric: 0.796875
Early stopping, best iteration is:
[53]	valid's binary_logloss: 0.411312	valid's Opt metric: 0.790323

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.355774	valid's Opt metric: 0.824427
Early stopping, best iteration is:
[83]	valid's binary_logloss: 0.352013	valid's Opt metric: 0.818182

===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_1_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's binary_logloss: 0.39042	valid's Opt metric: 0.796992
Early stopping, best iteration is:
[74]	valid's binary_logloss: 0.384343	valid's Opt metric: 0.793893

===== Start working with fold 3 for Lvl_0_

In [76]:
sample['Survived'] = (test_pred.data[:, 0] > 0.5).astype(int)
sample.to_csv('automl_utilized_600_f1_score.csv', index = False)

In [77]:
!kaggle competitions submit -c titanic -f /content/data/automl_utilized_600_f1_score.csv -m "automl"

100% 2.77k/2.77k [00:02<00:00, 1.15kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster